# About: LDAP認証の設定

---

MoodleのLDAP認証プラグインの設定を行います。

## 概要

![処理の流れ](images/moodle-211-01.png)

設定手順の主な流れを以下に示します。

1. CA証明書を配置する
1. LDAPサーバの名前解決に関する設定を追加した`compose.yaml`を配置する
1. 新たな設定でコンテナを起動し直す
1. Moodleの管理画面でLDAPサーバプラグインの設定を行う

上記の手順のうち1〜3はLDAPサーバとのTLS接続にプライベート認証局の証明書を用いている場合のみ必要な設定手順となります。LDAPサーバとの接続にTLSを利用しない場合、あるいはパブリックな認証局により発行された証明書を用いる場合は、手順4のみを実行してください。

## LDAPサーバに接続するための設定

LDAPサーバに接続するために必要となる設定を行います。

ここでの設定は、以下に示す条件に全て合致している場合のみ必要となります。

* LDAPサーバとの接続にTLSを利用している
* プライベート認証局により発行された証明書を利用している

該当しない場合はこの章をスキップして次に進んでください。

### グループ名の指定

操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定されたグループ名をチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

対象となるホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

### CA証明書の配置

LDAPサーバの証明書がプライベート認証局で発行されたものの場合、そのCA証明書をMoodle構築環境に配置します。

> ここで指定するCA証明書は事前にこのNotebook環境にアップロードしておいてください。

CA証明書のパスを指定してください。

In [ ]:
# (例)
# cacert_path = './certs/ldap-ca.crt'

cacert_path = 

MoodleコンテナにCA証明書を配置します。

In [ ]:
!ansible {target_group} -b -m copy \
    -a 'src={cacert_path} dest=/srv/moodle/moodle/conf/ca-trust/'

配置したCA証明書をMoodleコンテナのトラストアンカーに追加します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose exec moodle \
    update-ca-trust extract'

### LDAPサーバの名前解決を設定する

TLS接続を行う場合、LDAPサーバの名前解決が正しく行われる必要があります。ここではMoodleコンテナでの名前解決に関する設定を行います。

Moodleコンテナでのホストの名前解決を設定するには２つの方法があります。

* コンテナの `/etc/hosts` にエントリを追加する
* DNSサーバの設定を追加する

それぞれの方法に対応した設定を以下で行います。

#### コンテナの/etc/hostsにエントリを追加する

次のセルにホスト名とIPアドレスのペアを指定してください。キーにホスト名(FQDN)を、バリューにIPアドレスを指定します。`/etc/hosts`へのエントリ追加が不要な場合は実行せずに次に進んでください。

In [ ]:
# (例)
# moodle_extra_hosts = {
#    "ldap.example.org": "192.168.2.200",
# }

moodle_extra_hosts = {
    
}

#### DNSサーバの設定を追加する

次のセルにDNSサーバのアドレスを指定してください。DNSサーバの設定が不要な場合は実行せずに次に進んでください。

In [ ]:
# (例)
# moodle_dns_list = [
#      "192.168.1.1",  
# ]

moodle_dns_list = [
    
]

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py

if 'moodle_extra_hosts' in vars() and len(moodle_extra_hosts) > 0:
    update_group_vars(
        target_group,
        _file="30-ldap.yml",
        moodle_extra_hosts=moodle_extra_hosts,
    ) 
else:
    if 'moodle_extra_hosts' in load_group_vars(target_group):
        remove_group_vars(target_group, moodle_extra_hosts)    

if 'moodle_dns_list' in vars() and len(moodle_dns_list) > 0:
    update_group_vars(
        target_group,
        _file="30-ldap.yml",
        moodle_dns_list=moodle_dns_list,
    ) 
else:
    if 'moodle_dns_list' in load_group_vars(target_group):
        remove_group_vars(target_group, moodle_dns_list)    

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-ldap.yml

#### compose.yaml の配置

`/etc/hosts`またはDNSサーバの設定を変更した`compose.yaml`を配置し、設定を反映するためにコンテナを起動し直します。

実際に`compose.yaml`を配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -CDv -m template \
    -a 'src=template/compose.yaml.j2 \
    dest=/srv/moodle/compose.yaml backup=yes'

`compose.yaml`を構築環境に配置します。

In [ ]:
!ansible {target_group} -m template \
    -a 'src=template/compose.yaml.j2 \
    dest=/srv/moodle/compose.yaml backup=yes'

コンテナ設定の変更反映をするためにコンテナを起動し直します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up --wait --remove-orphans'

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

## LDAP認証プラグインの設定

MoodleでLDAP認証行うための設定を行います。

### LDAP認証プラグインの有効化

まずLDAP認証プラグインを有効にします。管理者ユーザでログインして[サイト管理]-[プラグイン]-[認証管理]を選択し認証プラグインの一覧を表示してください。以下のような画面が表示されます。

![認証管理画面](images/moodle-211-02.png)

図の「LDAPサーバ」の行の「有効にする」のアイコンを選択してLDAP認証プラグインを有効にしてください。

### LDAPサーバの設定

「LDAPサーバ」の「設定」のリンクを選択すると以下のような設定画面が表示されます。

![ldap設定画面](images/moodle-211-03.png)

「ホストURL」などにLDAPサーバのアドレスなどを設定してください。また同じ画面にある「バインド設定」でLDAPサーバに接続するための識別名、パスワードを、「ユーザルックアップ設定」でコンテクストなどを設定してください。最後に設定画面の最下部にある「変更を保存する」ボタンを選択してください。

### LDAPサーバへの接続確認

認証プラグインの一覧画面にある「設定テスト」リンクを選択すると、現在の設定でLDAPサーバに接続できるかをテストすることができます。

![ldap設定テスト](images/moodle-211-04.png)

LDAPサーバへの接続が成功すると以下のような表示となります。

![ldap設定結果](images/moodle-211-05.png)